In [ ]:
# Question 4

In [ ]:
from numpy.random import rand
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import expon
from scipy.stats import poisson
import matplotlib.pyplot as plt
import statistics as st
import pandas as pd

In [ ]:
# Stochastic model of both protein and mRNA (figure 7,9,10)

In [ ]:
#Set the values of the rates
k0 = 0.02
k1 = 0.01
k2 = 5
k3 = 1
# Set the stoichiometries of the reaction
stoichiometry = [1,-1]
# Define the propensities of the mRNA (defined as x)
def propensities_x(x):
    return[k0,k1*x] 
# Define the propensities of the protein (defined as y). Because production of y is dependant on x, both values are nessacary
def propensities_y(x,y):
    return[k2*x,k3*y]
# Now define the function for the reaction times of x. This function describes how the value of x changes over time
def reaction_times_x(x):
    a = propensities_x(x)
    aInv = [1/s if s>0 else np.inf for s in a]
    return -np.log(rand(2)) * aInv
reaction_times_x(1000)
# Now define the function for the reaction times for y. Once more, because production of y is dependant on x, both are
#nessacary for this function. 
def reaction_times_y(x,y):
    a = propensities_y(x,y)
    aInv = [1/s if s>0 else np.inf for s in a]
    return -np.log(rand(2)) * aInv
reaction_times_y(1000,1000)

#We now define the function for the ssa_step for mRNA (x). 
def ssa_step_x(x,tIn,tOut):
    t = tIn
    while t<tOut:
        rt = reaction_times_x(x)
        idx = np.argmin(rt)
        tau = np.min(rt)
        x+= stoichiometry[idx]
        t+= tau
    return x
#We now define the function for the ssa_step for Protein (y). 
def ssa_step_y(y,tIn,tOut):
    t = tIn
    while t<tOut:
        rt = reaction_times_y(x,y)
        idx = np.argmin(rt)
        tau = np.min(rt)
        y+= stoichiometry[idx]
        t+= tau
    return y
# We now set the starting values
dt = 1
x= 20
t = 0.0
y = 0
# The protein is given it's own vector of [y]
mRNA = [x]
time = [t]
protein = [y]
# Now over a 1000 points, the stochastic model appends mRNA, time, and protein arrays with x,t, and y
for i in range(1000):
    x = ssa_step_x(x,t,t+dt)
    y = ssa_step_y(y,t,t+dt)
    t += dt
    mRNA.append(x)
    protein.append(y)
    time.append(t)


In [ ]:
# The results are then plotted. 
fig = plt.figure(figsize = (20,8))
ax = fig.add_subplot(1, 1,1)
ax.plot(time, protein, color='tab:blue',label='Protein')
ax.plot(time, mRNA, color='tab:red',label='mRNA')
plt.legend()
ax.set_xlabel('time (s)')
ax.set_ylabel('Number of mRNA')
plt.title('Number of protein and mRNA over time(s)')

plt.show()


print("The mean of mRNA is", np.mean(mRNA), "The variance of mRNA is", np.var(mRNA))
print("The mean of protein is", np.mean(protein), "The variance of protein is", np.var(protein))
print("The Fano factor of mRNA is", np.var(mRNA)/np.mean(mRNA))
print("The Fano factor of protein is", np.var(protein)/np.mean(protein))

In [ ]:
# Histogram of protein Fano factors over a 1000 time points (figure 8.)

In [ ]:
from tqdm import tqdm # This is a computationally taxing piece of code. Use tqdm to track how long the script will take.
# Set the values of the rates
k0 = 0.2
k1 = 0.01
k2 = 5
k3 = 1

# Set the stoichiometries of the reaction
stoichiometry = [1, -1]

# Define the propensities of the mRNA (defined as x)
def propensities_x(x):
    return [k0, k1 * x]

# Define the propensities of the protein (defined as y). Both values x and y values are necessary.
def propensities_y(x, y):
    return [k2 * x, k3 * y]

# Now define the function for the reaction times of x.
def reaction_times_x(x):
    a = propensities_x(x)
    a_inv = [1/s if s > 0 else np.inf for s in a]
    return -np.log(np.random.rand(2)) * a_inv

# Now define the function for the reaction times for y.
def reaction_times_y(x, y):
    a = propensities_y(x, y)
    a_inv = [1/s if s > 0 else np.inf for s in a]
    return -np.log(np.random.rand(2)) * a_inv

# We now define the function for the ssa_step for mRNA (x).
def ssa_step_x(x, t_in, t_out):
    t = t_in
    while t < t_out:
        rt = reaction_times_x(x)
        idx = np.argmin(rt)
        tau = np.min(rt)
        x += stoichiometry[idx]
        t += tau
    return x

# We now define the function for the ssa_step for Protein (y).
def ssa_step_y(x, y, t_in, t_out):
    t = t_in
    while t < t_out:
        rt = reaction_times_y(x, y)
        idx = np.argmin(rt)
        tau = np.min(rt)
        y += stoichiometry[idx]
        t += tau
    return y

# Define a vector for the Fano factor
Fano = []

# Define the outer loop and starting values.
for j in tqdm(range(1000)):
    x = 20
    y = 100
    t = 0.0
    dt = 1

    # Set the vectors for mRNA, protein, and time
    mRNA = [x]
    protein = [y]
    time = [t]

    # In the internal loop, calculate reaction times and update values
    for i in range(1000):
        rt_x = reaction_times_x(x)
        idx_x = np.argmin(rt_x)
        tau_x = np.min(rt_x)
        x = ssa_step_x(x, t, t + dt)

        rt_y = reaction_times_y(x, y)
        idx_y = np.argmin(rt_y)
        tau_y = np.min(rt_y)
        y = ssa_step_y(x, y, t, t + dt)  # Corrected

        t += dt  # Corrected
        mRNA.append(x)
        protein.append(y)
        time.append(t)


    # Calculate Fano Factor
    protein_mean = np.mean(protein)
    protein_var = np.var(protein)
    Fano.append(protein_var / protein_mean)

# Plot the results on a histogram
plt.figure()
plt.hist(Fano, bins=20, color='c', edgecolor='k', alpha=0.65)
plt.axvline(np.mean(Fano), color='k', linestyle='dashed', linewidth=1)
min_ylim, max_ylim = plt.ylim()
plt.text(np.mean(Fano)*1.1, max_ylim*0.9, 'Mean: {:.2f}'.format(np.mean(Fano)))
Fano_mean = np.mean(Fano)
plt.title("Distribution of protein Fano Factor means over 1000 seconds")
Fano_var = np.var(Fano)

print("The Fano mean is:", Fano_mean, "The Fano variance is", Fano_var)
plt.show()